In [7]:
import yfinance as yf
import pandas as pd 
import os

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  

# Set up the Chrome driver using WebDriver Manager
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
wait = WebDriverWait(driver, 20)

##### Choose the index you want to scrape. Available are "dax", "mdax" and "sdax". 

In [9]:
index = "sdax"

In [8]:
if index == "ddax":
    url = "XETR-DAX"
elif index == "mdax":
    url = "XETR-MDAX"
elif index == "sdax":
    url = "XETR-SDXP"

# Navigate to the TradingView MDAX page
url = "https://www.tradingview.com/symbols/" + index + "/components/"
driver.get(url)

try:
    # Wait for the table to load using a more reliable selector
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".table-Ngq2xrcG")))

    # Extract table rows
    rows = driver.find_elements(By.CSS_SELECTOR, ".table-Ngq2xrcG tbody tr")

    # Extract data from rows
    data = []
    for row in rows:  # No need to skip header row since we are selecting tbody directly
        ticker_cell = row.find_element(By.CSS_SELECTOR, "td span.tickerCell-GrtoTeat")
        symbol = ticker_cell.find_element(By.CSS_SELECTOR, "a.tickerNameBox-GrtoTeat").text.strip()  # Extract ticker symbol
        name = ticker_cell.find_element(By.CSS_SELECTOR, "sup.tickerDescription-GrtoTeat").text.strip()  # Extract company name
        
        data.append({"Symbol": symbol, "Name": name})

except Exception as e:
    print(f"An error occurred: {e}")
    print(driver.page_source)  # Print page source for debugging

finally:
    # Close the browser
    driver.quit()

# Create a DataFrame from the extracted data if data is not empty
if data:
    df = pd.DataFrame(data)
    
    # Ensure the 'data' directory exists, create if it doesn't
    os.makedirs('data', exist_ok=True)
    
    # Save to CSV file in the 'data' subdirectory
    df.to_csv("data/" + index + ".csv", index=False)
    print("Data saved to data/" + index + ".csv")
else:
    print("No data extracted.")

Data saved to data/mdax.csv


In [6]:
# Load the CSV file
" + index + "mdax_df = pd.read_csv('data/" + index + ".csv')

# Determine the correct column names
symbol_column = 'symbol'
company_column = 'company'

# Create a list of company data
company_data_list = []

In [7]:
def get_company_data(symbol):
    ticker = yf.Ticker(symbol)
    data = ticker.info
    financials = ticker.financials

    latest_financials = financials.iloc[:, 0] if not financials.empty else pd.Series()

    price = data.get('currentPrice', None)
    market_cap = data.get('marketCap', None)
    revenue = latest_financials.get('Total Revenue', None)
    operating_income = latest_financials.get('Operating Income', None)
    net_income = latest_financials.get('Net Income', None)
    book_value = data.get('bookValue', None)

    try:
        cash_flow = latest_financials.get('Operating Cash Flow', None)
        if cash_flow is None:
            cash_flow = net_income + latest_financials.get('Depreciation', 0)
    except:
        cash_flow = None

    shares_outstanding = data.get('sharesOutstanding', None)
    kgv = price / (net_income / shares_outstanding) if all(v is not None and v != 0 for v in [price, net_income, shares_outstanding]) else None
    kuv = market_cap / revenue if all(v is not None and v != 0 for v in [market_cap, revenue]) else None
    kbv = price / book_value if all(v is not None and v != 0 for v in [price, book_value]) else None
    kcv = market_cap / cash_flow if all(v is not None and v != 0 for v in [market_cap, cash_flow]) else None

    return {
        'Symbol': symbol,
        'Price': price,
        'Market Cap': market_cap,
        'Revenue': revenue,
        'Operating Income': operating_income,
        'Net Income': net_income,
        'KGV (P/E)': kgv,
        'KUV (P/S)': kuv,
        'KBV (P/B)': kbv,
        'KCV (P/CF)': kcv
    }


# Iterate through each row in the CSV
for _, row in mdax_df.iterrows():
    symbol = row[symbol_column].strip()  # Remove any leading/trailing whitespace
    company_name = row[company_column].strip()  # Remove any leading/trailing whitespace
    print(f"Processing: {company_name} ({symbol})")  # Add this line for debugging
    company_data = get_company_data(symbol)
    company_data['Company'] = company_name
    company_data_list.append(company_data)

# Create a dataframe with the company data
result_df = pd.DataFrame(company_data_list)

# Reorder columns to put Symbol and Company first
columns_order = ['Symbol', 'Company'] + [col for col in result_df.columns if col not in ['Symbol', 'Company']]
result_df = result_df[columns_order]

# Print the result
result_df.head(5)



Processing: TALANX AG (TLX)
Processing: TRATON SE (8TRA)


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/8TRA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=8TRA&crumb=K0OqfItpey7


Processing: KNORR-BREMSE AG (KBX)
Processing: NEMETSCHEK SE (NEM)
Processing: HOCHTIEF AG (HOT)
Processing: HELLA GMBH & CO. KGAA (HLE)
Processing: RATIONAL AG (RAA)
Processing: DEUTSCHE WOHNEN SE (DWNI)


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/DWNI?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=DWNI&crumb=K0OqfItpey7


Processing: CTS EVENTIM AG & CO. KGAA (EVD)
Processing: DELIVERY HERO SE (DHER)


...

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/UTDI?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=UTDI&crumb=K0OqfItpey7


Processing: JUNGHEINRICH AG (JUN3)


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/JUN3?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=JUN3&crumb=K0OqfItpey7


Processing: THYSSENKRUPP AG (TKA)
Processing: REDCARE PHARMACY NV (RDC)
Processing: GERRESHEIMER AG (GXI)
Processing: TAG IMMOBILIEN AG (TEG)
Processing: K+S AG (SDF)


,Symbol,Company,Price,Market Cap,Revenue,Operating Income,Net Income,KGV (P/E),KUV (P/S),KBV (P/B),KCV (P/CF)
0,TLX,TALANX AG,15.5205,5.195084e+09,4.966590e+08,39236000.0,5.211000e+06,996.945661,10.460062,12.837469,996.945649
1,8TRA,TRATON SE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KBX,KNORR-BREMSE AG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NEM,NEMETSCHEK SE,40.6200,4.624384e+10,1.181200e+10,708000000.0,-2.494000e+09,-18.542037,3.914988,1.563992,-18.542037
4,HOT,HOCHTIEF AG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# Create the 'data' subdirectory if it doesn't exist
os.makedirs('data', exist_ok=True)

# Save the DataFrame to a CSV file in the 'data' subdirectory
csv_filename = 'data/mdax_results.csv'
df.to_csv(csv_filename, index=False)
print(f"Data has been saved to {csv_filename}")


Data has been saved to data/mdax.csv
